In [1]:
import pandas as pd
import numpy as np
import elapid
from elapid import MaxentModel
import xarray as xr
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, balanced_accuracy_score

In [2]:
X_train = pd.read_csv("/cluster/home/maikents/sinmod_features_masters/ready_for_training/X_train.csv")
X_test = pd.read_csv("/cluster/home/maikents/sinmod_features_masters/ready_for_training/X_test.csv")
y_train = pd.read_csv("/cluster/home/maikents/sinmod_features_masters/ready_for_training/y_train.csv")
y_test = pd.read_csv("/cluster/home/maikents/sinmod_features_masters/ready_for_training/y_test.csv")

In [ ]:
model = elapid.MaxentModel()
param_grid = {
    'beta_multiplier': [0.5, 1.0, 2.0, 3.0, 4.0],
    'beta_hinge': [0.5, 1.0, 1.5, 2.0],
    'beta_threshold': [0.5, 1.0, 1.5, 2.0]
}

#Perform Grid Search CV
cv_model = GridSearchCV(
    model, param_grid, 
    scoring={
        'ROC': make_scorer(roc_auc_score, needs_proba=True)
    }, 
    refit='ROC', 
    cv=10,
    return_train_score=True
)
cv_model.fit(X_train, y_train)

#Evaluate the best model on the test set
best_model = cv_model.best_estimator_

y_test_pred = best_model.predict_proba(X_test)[:, 1]

auc_score = roc_auc_score(y_test, y_test_pred)

print(f"Test AUC: {auc_score:.3f}")

/cluster/home/maikents/PyEnvCoralMapping/lib/python3.12/site-packages/sklearn/metrics/_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
/cluster/home/maikents/PyEnvCoralMapping/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:1000: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/cluster/home/maikents/PyEnvCoralMapping/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 139, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/cluster/home/maikents/PyEnvCoralMapping/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 371, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/cluster/home

In [ ]:
print("Best regularization parameters found:", cv_model.best_params_)

In [ ]:
cv_model.fit(X_train, y_train)
train_loss = 1 - cv_model.cv_results_['mean_train_ROC']
val_loss = 1 - cv_model.cv_results_['mean_test_ROC']

In [ ]:
best_model = cv_model.best_estimator_


In [ ]:
chunk_size = 2000
num_chunks = int(np.ceil(len(X_grid) / chunk_size))  

predicted_probabilities_list = []

for i in range(num_chunks):
    start = i * chunk_size
    end = min((i + 1) * chunk_size, len(X_grid))  
    X_chunk = X_grid.iloc[start:end]
    
    chunk_probabilities = best_model.predict_proba(X_chunk)[:, 1]
    
    predicted_probabilities_list.append(chunk_probabilities)

predicted_probabilities = np.concatenate(predicted_probabilities_list)


In [ ]:
num_x = 100128072
  # Number of unique x-coordinates
num_y = 100128072
  # Number of unique y-coordinates


# Reshape predicted_probabilities into the shape of the grid
# Assuming the grid is filled row by row (or column by column)
predicted_probabilities_grid = predicted_probabilities.reshape((num_y, num_x))


In [ ]:
np.save('/cluster/home/maikents/sinmod_features_masters/predicted_probabilities.npy', predicted_probabilities)


In [ ]:
print()len()